In [6]:
import json
import requests

API_URL = 'https://www.clubhouseapi.com/api'

PHONE_NUMBER = 'PHONE_NUMBER_WITH_A_PLUS_AND_THE_COUNTRY_CODE'
UUID = 'GENERATED_UUID'

HEADERS = {
    'User-Agent': 'clubhouse/269 (iPhone; iOS 14.1; Scale/3.00)',
    'CH-Languages': 'en-US',
    'CH-Locale': 'en_US',
    'CH-AppVersion': '0.2.15',
    'CH-AppBuild': '269',
    'CH-DeviceId': UUID.upper(),
}

def request(request_function, endpoint, *args):
    res = request_function(API_URL + endpoint, headers=HEADERS, *args)
    try:
        data = json.loads(res.text, encoding='UTF-8')
    except:
        data = res.text
    return [res.status_code, data]

- I created a temporary function `request` to easily send API requests to the server.
- Provide one UUID generated by `uuid.uuid4()`.
- Now, let's try logging in to Clubhouse.

In [10]:
def start_phone_number_auth():
    endpoint = '/start_phone_number_auth'
    print(API_URL + endpoint)
    res = request(requests.post, endpoint, {
        'phone_number': PHONE_NUMBER
    })
    print(res)

In [9]:
def call_phone_number_auth():
    endpoint = '/call_phone_number_auth'
    print(API_URL + endpoint)
    res = request(requests.post, endpoint, {})
    print(res)

In [1]:
def complete_phone_number_auth():
    endpoint = '/complete_phone_number_auth'
    print(API_URL + endpoint)
    res = request(requests.post, endpoint, {
        "verification_code": "8537",
        "phone_number": PHONE_NUMBER
    })
    print(res)

🎉 Yay! If you call the functions above, you'll have the auth token.

```python
 {
     'success': True,
     'is_verified': True,
     'user_profile': {
         'user_id': 4075733,
         'name': 'Junho Yeo',
         'photo_url': 'https://clubhouseprod.s3.amazonaws.com:443/4075733_c2e4e7ce-f37c-42be-951a-96aaddd20bfa_thumbnail_250x250',
         'username': 'junhoyeo'
     },
     'auth_token': 'AUTH_TOKEN',
     'refresh_token': 'REFRESH_TOKEN',
     'access_token': 'ACCESS_TOKEN',
     'is_waitlisted': False,
     'is_onboarding': False
}
```

In [3]:
user_id = 4075733
auth_token = 'AUTH_TOKEN'

def authenticated_request(request_function, endpoint, *args):
    res = request_function(API_URL + endpoint, headers=dict({
        'CH-UserID': str(user_id),
        'Authorization': f'Token {auth_token}'
    }, **HEADERS), *args)
    try:
        data = json.loads(res.text)
    except:
        data = res.text
    return [res.status_code, data]

I created another function, `authenticated_request` for sending authenticated API requests. This time, the headers includes your `user_id` and `auth_token`.

In [8]:
res = authenticated_request(requests.post, '/me', {})

In [7]:
from pprint import pprint

status, data = authenticated_request(requests.post, '/get_profile', {
    'user_id': user_id,
})
profile = data['user_profile']
impressive_keys = [
    'user_id', 'name', 'username', 'displayname', 'photo_url',
    'num_followers', 'num_following'
]

value_from_keys = {}
for key in impressive_keys:
    value_from_keys[key] = profile[key]

pprint(value_from_keys)
pprint(profile['invited_by_user_profile'])

{'displayname': None,
 'name': 'Junho Yeo',
 'num_followers': 103,
 'num_following': 98,
 'photo_url': 'https://clubhouseprod.s3.amazonaws.com:443/4075733_c2e4e7ce-f37c-42be-951a-96aaddd20bfa',
 'user_id': 4075733,
 'username': 'junhoyeo'}
{'name': 'Sunghyun Cho',
 'photo_url': 'https://clubhouseprod.s3.amazonaws.com:443/4061302_cf7d28ec-ce7e-4187-80b0-962e76247dc9_thumbnail_250x250',
 'user_id': 4061302,
 'username': 'anaclumos'}


😎 Cool, now you can get everyone's profile in Clubhouse!